# Pathway A

## Overall units:
   energy: GWh
   capacity: GW
   area: km^2
   money: €

## Assumptions from task description

In [15]:
# Define customers
customers = ['Customer_1_Steel_Plant', 'Customer_2_Chemical_Plant', 'Customer_3_Airport']

# Define time horizon
time_horizon = range(1, 11)  # Time periods from 1 to 10

# Define selling prices
price_per_unit = {
    'hydrogen': 210000,
    'ammonia': 287004,
    'jetfuel': 315000,
}

# Define typical capacity per unit area in GW/km^2
capacity_per_unit = {
    'photovoltaic': 20,
    'wind': 30,
}

CO2_demand_per_unit_jetfuel = 260 

capex = {
    'photovoltaic': 650 * 10 ** 6,
    'wind': 1500 * 10 ** 6,
    'PEM_electrolyzer': 700 * 10 ** 6,
    'alkaline_electrolyzer': 650 * 10 ** 6,
    'FT_synthesis': 1200 * 10 ** 6,
    'ammonia_synthesis': 1400 * 10 ** 6,
    'ammonia_splitting': 1000 * 10 **6,
    'battery': 550 * 10 **6, #in €/GWh
}

opex = {
    'photovoltaic': 0.03 * capex['photovoltaic'],
    'wind': 0.04 * capex['wind'],
    'PEM_electrolyzer': 0.04 * capex['PEM_electrolyzer'],
    'alkaline_electrolyzer': 0.04 * capex['alkaline_electrolyzer'],
    'FT_synthesis': 0.05 * capex['FT_synthesis'],
    'ammonia_synthesis': 0.05 * capex['ammonia_synthesis'],
    'ammonia_splitting': 0.05 * capex['ammonia_splitting'],
    'battery': 0.01 * capex['battery'], #Zahlen einfügen
}

transport_costs = {
    'hydrogen': 20 * 10**3, 
    'ammonia': 10 * 10**3,
    'jetfuel': 5 * 10**3,
}

point_source_costs = 70 

# Define the CO2 point source availabilty in tCO2 /a
#m = - 150000 / (2050-2023)
#b = 150000 

#point_source_availability = {}
#for t in time_horizon:
    #point_source_availability[t-1] = m*(t-1) + b
   # if t > 27:
        #point_source_availability[t-1] = 0

## Our own assumptions

In [16]:
# Decision variables
x = {
    'photovoltaic': 1,              # Do you want to use PV..
    'wind': 0,                      # or do you want to use wind?
    'PEM_electrolyzer': 1,          # Do you want to use PEM..
    'alkaline_electrolyzer': 0,     # or do you want to use Alkaline?
    'FT_synthesis': 1,
    'ammonia_synthesis': 1,
    'ammonia_splitting': 0,
    'battery': 1,
    }

x_transport = {
    'hydrogen': 1,                  # Do you want to produce hydrogen in morocco?
    'ammonia': 0,                   # Do you want to produce ammonia in morocco?
    'jetfuel': 0,                    # Do you want to produce jetfuel in morocco?
}

# Define supply to customers
y = {}
y['Customer_1_Steel_Plant'] = {}
y['Customer_2_Chemical_Plant'] = {}
y['Customer_3_Airport'] = {}

for t in time_horizon:
    y['Customer_1_Steel_Plant'][t-1] = 1000
    y['Customer_2_Chemical_Plant'][t-1] = 500
    y['Customer_3_Airport'][t-1] = 500

# Define the size of technologies in GW
capacity = {
    'photovoltaic': 1,
    'wind': 0,
    'PEM_electrolyzer': 1,
    'alkaline_electrolyzer': 0,
    'FT_synthesis': 1,
    'ammonia_synthesis': 1,
    'ammonia_splitting':0,
    'battery': 1, #in GWh
}

# Discount rate
i = 0.1

print(y)

{'Customer_1_Steel_Plant': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 'Customer_2_Chemical_Plant': {0: 500, 1: 500, 2: 500, 3: 500, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, 9: 500}, 'Customer_3_Airport': {0: 500, 1: 500, 2: 500, 3: 500, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, 9: 500}}


### Assumption Check

In [17]:
if(x['wind'] + x['photovoltaic']) > 1: print('Caution, both wind and PV selected')
if(x['PEM_electrolyzer'] + x['alkaline_electrolyzer']) > 1: print('Caution, both PEM and alkaline electrolyzers selected')
if y['Customer_1_Steel_Plant'][t-1] > 0 and x_transport['hydrogen'] < 1: print('Caution, is hydrogen split in rotterdam?')
if y['Customer_2_Chemical_Plant'][t-1] > 0 and x_transport['ammonia'] < 1: print('Ammonia is split in rotterdam?')
if y['Customer_3_Airport'][t-1] > 0 and x_transport['jetfuel'] < 1: print('Jet Fuel is split in rotterdam?')
if (x['ammonia_splitting'] + x_transport['hydrogen']) > 1: print('If hydrogen is transported there is no need for ammonia splitting')

Ammonia is split in rotterdam?
Jet Fuel is split in rotterdam?


## Let's start the NPV calculation

In [18]:
# Compute transportedd amounts in GWh
transported_product = {
    'ammonia':  [y['Customer_2_Chemical_Plant'][t-1] * x_transport['ammonia'] + (y['Customer_1_Steel_Plant'][t-1] / 0.7) * x['ammonia_splitting'] + (y['Customer_3_Airport'][t-1] / (0.7*0.75*0.71)) * x['ammonia_splitting']for t in time_horizon], 
    'jetfuel': [y['Customer_3_Airport'][t-1] * x_transport['jetfuel'] for t in time_horizon],
    'hydrogen': [y['Customer_1_Steel_Plant'][t-1] * x_transport['hydrogen'] + y['Customer_2_Chemical_Plant'][t-1] * (1- x_transport['ammonia']) / 0.8 + y['Customer_3_Airport'][t-1] * (1- x_transport['jetfuel']) / (0.71 * 0.75)  for t in time_horizon]
    #'hydrogen': [y['Customer_1_Steel_Plant'][t-1] * x_ammoniak_splitting / efficiency_ammoniak_splitting + [y['Customer_1_Steel_Plant'][t-1] * x_transport['hydrogen'] + y['Customer_2_Chemical_Plant'][t-1] * (1- x_transport['ammonia']) / 0.8 + y['Customer_3_Airport'][t-1] * (1- x_transport['jetfuel']) / (0.71 * 0.75)  for t in time_horizon]

}

technologies = ['photovoltaic', 'wind', 'PEM_electrolyzer', 'alkaline_electrolyzer', 'FT_synthesis', 'ammonia_synthesis','ammonia_splitting','battery']
products = ['hydrogen', 'ammonia', 'jetfuel']


cash_inflow_customer = {}

for c in customers:
    cash_inflow_customer[c] = {}
    for t in time_horizon:
        if c == 'Customer_1_Steel_Plant':
            cash_inflow_customer[c][t-1] = price_per_unit['hydrogen'] * y[c][t-1]
        elif c == 'Customer_2_Chemical_Plant':
            cash_inflow_customer[c][t-1] = price_per_unit['ammonia'] * y[c][t-1]
        elif c == 'Customer_3_Airport':
            cash_inflow_customer[c][t-1] = price_per_unit['jetfuel'] * y[c][t-1]
        

print(cash_inflow_customer)

# Initial Investment
#init_investment = sum(capex[i] * x[i] for i in technologies) ist doch auch auf die Kapazität bezogen
init_investment_tech = {}

for i in technologies:
    init_investment_tech[i] = capex[i] * capacity[i] 

print(init_investment_tech)

init_investment = sum(init_investment_tech[i] for i in technologies)

print(init_investment)

cash_inflow = {}
for t in time_horizon:
    cash_inflow[t-1] = sum(cash_inflow_customer[c][t-1] for c in customers for t in time_horizon)

print(cash_inflow)

#Robert angucken kann ich y['Customer_3_Airport'] so verwenden oder wie greife ich darauf zu?
cash_outflow_CO2 = {}

for t in time_horizon:
    cash_outflow_CO2 [t-1] = CO2_demand_per_unit_jetfuel * point_source_costs * y['Customer_3_Airport'][t-1]

print(cash_outflow_CO2)

cash_outflow_technology = {}

for i in technologies:
    cash_outflow_technology[i] = {}
    for t in time_horizon:
        cash_outflow_technology[i][t-1] = opex[i]  * capacity[i]

print(cash_outflow_technology)

cash_outflow_transport = {}
for i in products:
    cash_outflow_transport[i] = {}
    for t in time_horizon:
        cash_outflow_transport[i][t-1] = transported_product[i][t-1] * transport_costs[i]

print(cash_outflow_transport)

cash_outflow = {}
for t in time_horizon:
    cash_outflow[t-1] = sum(cash_outflow_technology[i][t-1] for i in technologies) + sum(cash_outflow_transport[i][t-1] for i in products) + cash_outflow_CO2[t-1]

{'Customer_1_Steel_Plant': {0: 210000000, 1: 210000000, 2: 210000000, 3: 210000000, 4: 210000000, 5: 210000000, 6: 210000000, 7: 210000000, 8: 210000000, 9: 210000000}, 'Customer_2_Chemical_Plant': {0: 143502000, 1: 143502000, 2: 143502000, 3: 143502000, 4: 143502000, 5: 143502000, 6: 143502000, 7: 143502000, 8: 143502000, 9: 143502000}, 'Customer_3_Airport': {0: 157500000, 1: 157500000, 2: 157500000, 3: 157500000, 4: 157500000, 5: 157500000, 6: 157500000, 7: 157500000, 8: 157500000, 9: 157500000}}
{'photovoltaic': 650000000, 'wind': 0, 'PEM_electrolyzer': 700000000, 'alkaline_electrolyzer': 0, 'FT_synthesis': 1200000000, 'ammonia_synthesis': 1400000000, 'ammonia_splitting': 0, 'battery': 550000000}
4500000000
{0: 5110020000, 1: 5110020000, 2: 5110020000, 3: 5110020000, 4: 5110020000, 5: 5110020000, 6: 5110020000, 7: 5110020000, 8: 5110020000, 9: 5110020000}
{0: 9100000, 1: 9100000, 2: 9100000, 3: 9100000, 4: 9100000, 5: 9100000, 6: 9100000, 7: 9100000, 8: 9100000, 9: 9100000}
{'photov

In [19]:
i = 0.1

NPV = - init_investment + sum(
    ( cash_inflow[t - 1]
    - cash_outflow[t - 1]
    ) 
    / ((1 + i) ** t)
    for t in time_horizon
)

print(NPV)

#ROI = ... / ...
#print(ROI)

#amortization = ...
#print(amortization)



25403400097.98923
